In [1]:
import pandas as pd
import numpy as np

In [2]:
def resumen(datos):
    return print('Forma:\n', datos.shape,'\n\n',
                 'Columnas:\n', datos.columns,'\n\n',
                 'Tipos:\n', datos.dtypes,'\n\n',
                 'Datos nulos:\n', datos.isnull().sum(),'\n\n',
                 'Cabecera:\n', datos.head(10),'\n\n',
                 'Últimos datos:\n', datos.tail(10),'\n\n',
                 'Estadísticos:\n', datos.describe([0.05,0.25,0.5,0.75,0.95])
                )
def truncar(numero, decimales = 0):
    return np.trunc(numero*10**decimales)/(10**decimales)

In [3]:
datos_para_subir = pd.read_csv(
    'C:\\Users\\nahue\\Desktop\\Tesis_2\\Datos\\Para_subir.txt',
    delimiter = '\t',
    decimal = '.',
    na_values = -9999,
    encoding = 'ascii'
    )

In [4]:
resumen(datos_para_subir)

Forma:
 (14444, 12) 

 Columnas:
 Index(['Year', 'DoY', 'Hour', 'NEE', 'LE', 'H', 'Rg', 'Tair', 'Tsoil', 'HR1',
       'VPD', 'Ustar'],
      dtype='object') 

 Tipos:
 Year     object
DoY      object
Hour     object
NEE      object
LE       object
H        object
Rg       object
Tair     object
Tsoil    object
HR1      object
VPD      object
Ustar    object
dtype: object 

 Datos nulos:
 Year        0
DoY         0
Hour        0
NEE      6285
LE       5888
H        5043
Rg       1392
Tair     1392
Tsoil    1392
HR1      1392
VPD      1392
Ustar    4472
dtype: int64 

 Cabecera:
      Year   DoY  Hour         NEE    LE     H                 Rg  \
0       -     -     -  umolm-2s-1  Wm-2  Wm-2               Wm-2   
1  2018.0  54.0   9.0         NaN   NaN   NaN              525.4   
2  2018.0  54.0   9.5         NaN   NaN   NaN             555.25   
3  2018.0  54.0  10.0         NaN   NaN   NaN  673.1499999999999   
4  2018.0  54.0  10.5         NaN   NaN   NaN  832.9999999999998   
5  20

In [97]:
cols = datos_para_subir.columns.drop('DoY')
datos_para_trabajar = datos_para_subir[1:].astype('float64')

faltantes = datos_para_trabajar[cols].isnull()
faltantes['DoY'] = datos_para_trabajar['DoY']
faltantes = faltantes.groupby('DoY',
                              as_index = False,
                              sort = False
                             ).sum().drop('DoY', 1)

medidos = datos_para_trabajar[cols].notnull()
medidos['DoY'] = datos_para_trabajar['DoY']
medidos = medidos.groupby('DoY',
                          as_index = False,
                          sort = False
                          ).sum().drop('DoY', 1)

In [98]:
dias = np.array([0, 16, 46, 76,
                107, 137, 168, 199,
                229, 260, 290, 303
                ])
lista = list(range(1, len(dias)))
mes_falt = faltantes[cols].iloc[0:len(dias)]
mes_med = medidos[cols].iloc[0:len(dias)]
for i in lista:
    desde = dias[i-1]
    hasta = dias[i]
    mes_falt.iloc[i-1] = faltantes.iloc[desde:hasta].sum()
    mes_med.iloc[i-1] = medidos.iloc[desde:hasta].sum()
mes_falt['Year'] = mes_med['Year']
mes_falt = mes_falt.drop('Hour', 1).rename(columns = {"Year": "Mediciones"})
mes_falt = mes_falt.drop(11, 0)

In [99]:
mes_falt.head(15)

,Mediciones,NEE,LE,H,Rg,Tair,Tsoil,HR1,VPD,Ustar
0,750.0,750.0,750.0,750.0,0.0,0.0,0.0,0.0,0.0,750.0
1,1440.0,1185.0,1166.0,1144.0,0.0,0.0,0.0,0.0,0.0,1133.0
2,1440.0,1403.0,1390.0,1343.0,0.0,0.0,0.0,0.0,0.0,1343.0
3,1488.0,566.0,514.0,428.0,859.0,859.0,859.0,859.0,859.0,385.0
4,1440.0,520.0,480.0,375.0,19.0,19.0,19.0,19.0,19.0,334.0
5,1488.0,768.0,690.0,602.0,87.0,87.0,87.0,87.0,87.0,486.0
6,1488.0,341.0,272.0,43.0,3.0,3.0,3.0,3.0,3.0,5.0
7,1440.0,273.0,229.0,129.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1488.0,141.0,115.0,74.0,0.0,0.0,0.0,0.0,0.0,3.0
9,1440.0,282.0,230.0,123.0,10.0,10.0,10.0,10.0,10.0,12.0


In [100]:
mes_falt.sum()

Mediciones    14443.0
NEE            6285.0
LE             5888.0
H              5043.0
Rg             1392.0
Tair           1392.0
Tsoil          1392.0
HR1            1392.0
VPD            1392.0
Ustar          4472.0
dtype: float64

In [121]:
noche = datos_para_trabajar[(datos_para_trabajar['Rg'] <= 10)]
dia = datos_para_trabajar[(datos_para_trabajar['Rg'] > 10)]
rg_nan = datos_para_trabajar[(datos_para_trabajar['Rg'].isnull())]

In [122]:
noche.isnull().sum()

Year        0
DoY         0
Hour        0
NEE      3557
LE       3378
H        2724
Rg          0
Tair        0
Tsoil       0
HR1         0
VPD         0
Ustar    2396
dtype: int64

In [123]:
dia.isnull().sum()

Year        0
DoY         0
Hour        0
NEE      2435
LE       2269
H        2158
Rg          0
Tair        0
Tsoil       0
HR1         0
VPD         0
Ustar    1940
dtype: int64

In [124]:
rg_nan.isnull().sum()

Year        0
DoY         0
Hour        0
NEE       293
LE        241
H         161
Rg       1392
Tair     1392
Tsoil    1392
HR1      1392
VPD      1392
Ustar     136
dtype: int64